In [0]:
client_id=""
tenant_id=""
client_secret=""

spark.conf.set("fs.azure.account.auth.type.storagedatafactory2605.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.storagedatafactory2605.dfs.core.windows.net",
               "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")

spark.conf.set("fs.azure.account.oauth2.client.id.storagedatafactory2605.dfs.core.windows.net", client_id)
spark.conf.set("fs.azure.account.oauth2.client.secret.storagedatafactory2605.dfs.core.windows.net", client_secret)

spark.conf.set("fs.azure.account.oauth2.client.endpoint.storagedatafactory2605.dfs.core.windows.net",
               f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")


In [0]:
df = spark.read.format("delta").load(
    "abfss://fgold@storagedatafactory2605.dfs.core.windows.net/monthly_sales_enriched_delta/"
)

pdf = df.toPandas()
pdf = pdf.sort_values(["product_name", "order_ym"])


In [0]:
pdf.shape

(1818, 15)

In [0]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error
import warnings
warnings.filterwarnings("ignore")

def mape(y_true, y_pred):
    return mean_absolute_percentage_error(y_true, y_pred)


In [0]:
def train_test_split_ts(y, test_size=3):
    return y[:-test_size], y[-test_size:]


In [0]:
def model_naive(train):
    forecast = [train[-1]] * 3
    return forecast


In [0]:
def model_ma(train):
    if len(train) < 3:
        avg = np.mean(train)
        return [avg] * 3
    avg = np.mean(train[-3:])
    return [avg] * 3


In [0]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

def model_es(train):
    model = ExponentialSmoothing(train, trend="add", seasonal=None).fit()
    forecast = model.forecast(3)
    return forecast


In [0]:
from statsmodels.tsa.arima.model import ARIMA

def model_arima(train):
    model = ARIMA(train, order=(1,1,1)).fit()
    forecast = model.forecast(3)
    return forecast


In [0]:
%pip install prophet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/50.0 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/57.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/12.1 MB ? eta -:--:--
   ━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/12.1 MB 43.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/12.1 MB 66.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━ 8.7/12.1 MB 84.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 12.1/12.1 MB 118.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 12.1/12.1 MB 118.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/99.1 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 7.8 MB/s eta 0:00:00
   ━━

In [0]:
from prophet import Prophet

def model_prophet(train, dates):
    dfp = pd.DataFrame({"ds": dates, "y": train})
    model = Prophet()
    model.fit(dfp)

    future = model.make_future_dataframe(periods=3, freq="M")
    fc = model.predict(future)["yhat"].tail(3).values
    return fc


In [0]:
%pip install pmdarima


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/689.1 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 686.1/689.1 kB 20.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.1/689.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/10.4 MB ? eta -:--:--
   ━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/10.4 MB 42.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/10.4 MB 52.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━ 7.5/10.4 MB 71.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 10.4/10.4 MB 82.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 10.4/10.4 MB 82.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 56.2 MB/s eta 0:00:00
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.14.2
    Not uninstalling statsmodels at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs

In [0]:
from pmdarima import auto_arima


In [0]:
def model_arima(y_train):
    try:
        model = auto_arima(
            y_train,
            seasonal=False,
            trace=False,
            error_action='ignore',
            suppress_warnings=True
        )
        forecast = model.predict(n_periods=3)
        return forecast
    except:
        return np.repeat(y_train[-1], 3)


In [0]:
pdf.shape


(1818, 15)

In [0]:
def forecast_product(y, dates):
    """
    Returns: (best_model_name, forecast_value, mape_value)
    """

    n = len(y)

    # -------------------------
    # CASE 1 — Only 1 Month
    # -------------------------
    if n == 1:
        return "Naive-1M", y[-1], None

    # -------------------------
    # CASE 2 — 2–3 Months
    # -------------------------
    if n <= 3:
        return "MA-Short", np.mean(y), None

    # -------------------------
    # CASE 3 — 4–5 Months
    # -------------------------
    if n <= 5:
        model = ExponentialSmoothing(y).fit()
        pred = model.forecast(1)[0]
        return "ES-Medium", pred, None

    # -------------------------
    # CASE 4 — 6–7 Months
    # -------------------------
    if n <= 7:
        model = auto_arima(y, seasonal=False, suppress_warnings=True)
        pred = model.predict(n_periods=1)[0]
        return "ARIMA-Medium", pred, None

    # -------------------------
    # CASE 5 — ≥ 8 Months (Full Model Selection)
    # -------------------------
    train, test = train_test_split_ts(y, test_size=3)

    # 1) NAIVE
    pred_naive = model_naive(train)
    mape_naive = mape(test, pred_naive)

    # 2) MA
    pred_ma = model_ma(train)
    mape_ma = mape(test, pred_ma)

    # 3) ES
    pred_es = model_es(train)
    mape_es = mape(test, pred_es)

    # 4) ARIMA
    pred_arima = model_arima(train)
    mape_arima = mape(test, pred_arima)

    # 5) PROPHET
    pred_prophet = model_prophet(train, dates[:-3])
    mape_prophet = mape(test, pred_prophet)

    mape_list = [mape_naive, mape_ma, mape_es, mape_arima, mape_prophet]
    models = ["Naive", "MA", "ES", "ARIMA", "Prophet"]

    best_index = np.argmin(mape_list)
    best_model = models[best_index]

    # ------------------------------------------------
    # Retrain FULL model on all data for final forecast
    # ------------------------------------------------
    if best_model == "Naive":
        pred = y[-1]

    elif best_model == "MA":
        pred = np.mean(y[-3:])

    elif best_model == "ES":
        pred = ExponentialSmoothing(y).fit().forecast(1)[0]

    elif best_model == "ARIMA":
        pred = auto_arima(y, seasonal=False).predict(1)[0]

    else:  # PROPHET
        # model_prophet returns numpy array → no iloc
        pred = model_prophet(y, dates)[-1]   # FIXED

    return best_model, pred, mape_list[best_index]


In [0]:
def forecast_horizon(y, dates, best_model, horizon):
    """
    Forecast future months using the best model selected earlier.
    horizon = 1, 3, 6
    """
    y = np.array(y)

    if best_model == "Naive":
        return [y[-1]] * horizon

    elif best_model == "MA":
        return [np.mean(y[-3:])] * horizon

    elif best_model == "ES":
        model = ExponentialSmoothing(y).fit()
        return model.forecast(horizon)

    elif best_model == "ARIMA":
        model = auto_arima(y, seasonal=False, suppress_warnings=True)
        return model.predict(n_periods=horizon)

    elif best_model == "Prophet":
        df_prophet = pd.DataFrame({"ds": dates, "y": y})
        m = Prophet()
        m.fit(df_prophet)

        future = m.make_future_dataframe(periods=horizon, freq="M")
        forecast = m.predict(future)

        return list(forecast["yhat"].tail(horizon))

    else:
        return [None] * horizon


In [0]:
def forecast_fast(y, model_name, horizon):
    y = np.array(y)

    if model_name in ["Naive", "Naive-1M"]:
        return [y[-1]] * horizon
    

    if model_name in ["MA", "MovingAverage", "MA-Short"]:
        window = min(3, len(y))
        return [np.mean(y[-window:])] * horizon

    if model_name in ["ES", "ExponentialSmoothing", "ES-Medium"]:
        model = ExponentialSmoothing(y).fit()
        return model.forecast(horizon)

    if model_name in ["ARIMA", "ARIMA-Medium"]:
        model = auto_arima(y, seasonal=False, suppress_warnings=True)
        return model.predict(horizon)

    if model_name == "Prophet":
        df = pd.DataFrame({"ds": pd.date_range(start="2000-01-01", periods=len(y), freq="M"),
                           "y": y})
        m = Prophet()
        m.fit(df)
        future = m.make_future_dataframe(periods=horizon, freq="M")
        forecast = m.predict(future)
        return forecast["yhat"].tail(horizon).values

    return [y[-1]] * horizon


In [0]:
forecast_results = []

for p in pdf["product_name"].unique():

    df_p = pdf[pdf["product_name"] == p].sort_values("order_ym")
    y = df_p["monthly_sales"].values
    dates = pd.to_datetime(df_p["order_ym"])

    model_name, pred, mape_value = forecast_product(y, dates)

    forecast_results.append([
        p, len(y), model_name, pred, mape_value
    ])

df_forecast_all = pd.DataFrame(
    forecast_results,
    columns=["product_name", "history_length", "model_used", "forecast_next_month", "MAPE"]
)


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

🏃 View run caring-dolphin-540 at: https://adb-4421020624872303.3.azuredatabricks.net/ml/experiments/4241256389027932/runs/43a8bb26999a41e68bbec8934e9171d3
🧪 View experiment at: https://adb-4421020624872303.3.azuredatabricks.net/ml/experiments/4241256389027932


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

🏃 View run delightful-ray-184 at: https://adb-4421020624872303.3.azuredatabricks.net/ml/experiments/4241256389027932/runs/e164ae7bd6cb40f68aa33aeb32b6a89a
🧪 View experiment at: https://adb-4421020624872303.3.azuredatabricks.net/ml/experiments/4241256389027932


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

...

🏃 View run stylish-gnat-488 at: https://adb-4421020624872303.3.azuredatabricks.net/ml/experiments/4241256389027932/runs/cbba18b658e44354805ac1408d1a3f73
🧪 View experiment at: https://adb-4421020624872303.3.azuredatabricks.net/ml/experiments/4241256389027932


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

🏃 View run gifted-crane-857 at: https://adb-4421020624872303.3.azuredatabricks.net/ml/experiments/4241256389027932/runs/ed249a11f8544f7eab293d0f1499c66c
🧪 View experiment at: https://adb-4421020624872303.3.azuredatabricks.net/ml/experiments/4241256389027932


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

🏃 View run valuable-mouse-417 at: https://adb-4421020624872303.3.azuredatabricks.net/ml/experiments/4241256389027932/runs/c9e44d9417dc4e9381a8b9e3841d3cbd
🧪 View experiment at: https://adb-4421020624872303.3.azuredatabricks.net/ml/experiments/4241256389027932


In [0]:
from pyspark.sql import SparkSession

# Convert Pandas DF → Spark DF
df_forecast_spark = spark.createDataFrame(df_forecast_all)

gold_base = "abfss://fgold@storagedatafactory2605.dfs.core.windows.net/"

# Save as Delta
df_forecast_spark.write.format("delta").mode("overwrite").save(
    gold_base + "forecast_delta/"
)

print("Forecast saved to:", gold_base + "forecast_delta/")


Forecast saved to: abfss://fgold@storagedatafactory2605.dfs.core.windows.net/forecast_delta/


In [0]:
df_forecast_spark.write.mode("overwrite").parquet(
    "abfss://fgold@storagedatafactory2605.dfs.core.windows.net/export/forecast_results/"
)


In [0]:
df_forecast_all.display()

product_name,history_length,model_used,forecast_next_month,MAPE
adidas brazuca 2014 official match ball,6,ARIMA-Medium,1733.225059583334,null
adidas kids' f5 messi fg soccer cleat,28,ARIMA,1067.841326406601,0.4704365086021747
adidas men's f10 messi trx fg soccer cleat,28,ES,1934.88859940676,0.3667443395896548
adidas men's germany black crest away tee,28,Naive,725.0,0.16111111111111112
adidas youth germany black/red away match soc,28,ARIMA,2422.5,0.08178834182229765
adult dog supplies,3,MA-Short,13841.60025091999,null
baby sweater,2,MA-Short,6114.780189404997,null
bag boy beverage holder,28,ES,746.3788083760325,0.12737855000905007
bag boy m330 push cart,6,ARIMA-Medium,3386.8218812284467,null
bowflex selecttech 1090 dumbbells,2,MA-Short,2999.949951,null
